In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df_dup = pd.read_csv('/home/fahd/Documents/headline-trader/headline-trader/src/training/full_df.csv')

df = df_dup.drop_duplicates()


df

,timestamp,title,price,sentiment,sentiment_score,future_price,label,symbol,hour_of_day,day_of_week
0,2025-04-25 14:51:09.451590+00:00,Cryptocurrency campaigners call for Swiss cent...,95478.3600,neutral,0.854923,94933.0700,sell,BTCUSDT,14,4
1,2025-04-25 14:51:10.891138+00:00,The Fed Just Quietly Primed Bitcoin And Crypto...,95478.3600,neutral,0.902940,94933.0700,sell,BTCUSDT,14,4
2,2025-04-25 14:51:12.271174+00:00,Bitcoin ETFs Gain as Dollar Trust Falters - ET...,95478.3600,negative,0.740927,94933.0700,sell,BTCUSDT,14,4
3,2025-04-25 14:51:13.556977+00:00,Bitcoin – a certain kind of hedge? - DWS Asset...,95478.3600,neutral,0.939790,94933.0700,sell,BTCUSDT,14,4
4,2025-04-25 14:51:14.739492+00:00,Bitcoin gets complicated as Tether and SoftBan...,95478.3600,neutral,0.910740,94933.0700,sell,BTCUSDT,14,4
...,...,...,...,...,...,...,...,...,...,...
6718,2025-04-26 02:44:12.506789+00:00,Could XRP Revolutionize Global Finance by Repl...,2.1942,neutral,0.874792,2.1942,hold,XRPUSDT,2,5
6719,2025-04-26 02:44:14.402615+00:00,What XRP Army Expects from Today’s SEC Round T...,2.1942,neutral,0.943423,2.1942,hold,XRPUSDT,2,5
6720,2025-04-26 02:44:15.867894+00:00,XRP and Altcoins Under the Microscope: What To...,2.1942,neutral,0.908315,2.1942,hold,XRPUSDT,2,5
6721,2025-04-26 02:44:19.345406+00:00,Cryptocurrency prices in India today (26 Apr 2...,2.1942,neutral,0.860967,2.1942,hold,XRPUSDT,2,5


In [11]:
if 'change' not in df.columns:
    df['change'] = (df['future_price'] - df['price']) / df['price']

In [13]:
# Define the new threshold
new_threshold = 0.015

# Recompute labels using the new threshold
df['label'] = np.select(
    [df['change'] > new_threshold, df['change'] < -new_threshold],
    ['buy', 'sell'],
    default='hold'
)

print("DataFrame with Updated Labels:")
print(df[['title', 'future_price', 'price', 'change', 'label']].head(10))

DataFrame with Updated Labels:
                                               title  future_price     price  \
0  Cryptocurrency campaigners call for Swiss cent...      94933.07  95478.36   
1  The Fed Just Quietly Primed Bitcoin And Crypto...      94933.07  95478.36   
2  Bitcoin ETFs Gain as Dollar Trust Falters - ET...      94933.07  95478.36   
3  Bitcoin – a certain kind of hedge? - DWS Asset...      94933.07  95478.36   
4  Bitcoin gets complicated as Tether and SoftBan...      94933.07  95478.36   
5  Cathie Wood-Led Ark Invest Raises Bullish BTC ...      94933.07  95478.36   
6  Swiss National Bank chief dismisses Bitcoin re...      94933.07  95478.36   
7  Is 8% Of Bitcoin Owned By Institutions A Threa...      94933.07  95478.36   
8  CoinShares Q4 2024 Mining Report. Bitcoin vs G...      94933.07  95478.36   
9  Bitcoin whale moves 50 BTC after 15 years, mak...      94933.07  95478.36   

     change label  
0 -0.005711  hold  
1 -0.005711  hold  
2 -0.005711  hold  
3 -0.005

In [ ]:


label_map = {'buy': 0, 'hold': 1, 'sell': 2}
df['label'] = df['label'].map(label_map)


# Features & labels
features = df[['price', 'sentiment_score', 'hour_of_day', 'day_of_week']].values
labels = df['label'].values

features, labels

In [ ]:
# Normalize
scaler = StandardScaler()
features = scaler.fit_transform(features)


# convert to pytorch tensors
X = torch.tensor(features, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

X.shape[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
# Using DataLoader for batching 

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)


In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(32, 3)  # 3 classes: buy/sell/hold

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

model = MyModel(input_size=X.shape[1])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
epochs = 20
for epoch in range(epochs):
    
    # --- Training Phase ---
    model.train()  # important: sets model to training mode
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # --- Validation Phase ---
    model.eval()  # important: disables dropout, batchnorm, etc.
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # no gradient calculations
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    val_accuracy = correct / total
    print(f"Epoch {epoch}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")


In [ ]:
correct = 0
test_loss = 0
total = 0
criterion = nn.CrossEntropyLoss()

model.eval()
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")
print(f"Test Loss = {test_loss / len(test_loader):.4f}")

In [ ]:
torch.save(model.state_dict(), "model.pth")